# 🧹 Data Cleaning and Preprocessing

This notebook handles:
- Loading raw startup data from CSV/JSON files
- Cleaning and validating data
- Transforming data into MongoDB-compatible format
- Preparing data for ingestion into MongoDB Atlas

In [ ]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
import os
import sys

# Add parent directory to path for imports
sys.path.append('..')
import config

## 1. Load Raw Data

Load startup data from various sources (CSV, JSON, API)

In [ ]:
# Example: Load from CSV
def load_csv_data(filepath):
    """Load startup data from CSV file."""
    try:
        df = pd.read_csv(filepath)
        print(f"✅ Loaded {len(df)} records from {filepath}")
        return df
    except FileNotFoundError:
        print(f"❌ File not found: {filepath}")
        return None

# Example: Load from JSON
def load_json_data(filepath):
    """Load startup data from JSON file."""
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
        print(f"✅ Loaded {len(data)} records from {filepath}")
        return pd.DataFrame(data)
    except FileNotFoundError:
        print(f"❌ File not found: {filepath}")
        return None

# For demonstration, create sample data
sample_data = {
    'name': ['TechCorp', 'FinanceAI', 'HealthPlus', 'EduTech', 'GreenEnergy'],
    'founded_year': [2018, 2019, 2017, 2020, 2016],
    'country': ['USA', 'UK', 'Germany', 'India', 'USA'],
    'city': ['New York', 'London', 'Berlin', 'Bangalore', 'San Francisco'],
    'industry': ['SaaS,Enterprise', 'FinTech,AI', 'HealthTech', 'EdTech,AI', 'CleanTech,Energy'],
    'total_funding_usd': [5000000, 12000000, 8000000, 3000000, 15000000],
    'employee_count': [50, 120, 80, 30, 200],
    'status': ['Operating', 'Operating', 'Operating', 'Operating', 'Acquired']
}

df = pd.DataFrame(sample_data)
print("\n📊 Sample Data Loaded:")
df.head()

## 2. Data Exploration and Quality Checks

In [ ]:
# Basic info
print("📋 Dataset Info:")
print(f"Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nData Types:")
print(df.dtypes)

# Check for missing values
print("\n🔍 Missing Values:")
missing = df.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else "No missing values found!")

# Check for duplicates
duplicates = df.duplicated(subset=['name']).sum()
print(f"\n🔄 Duplicate company names: {duplicates}")

## 3. Data Cleaning and Transformation

In [ ]:
def clean_startup_data(df):
    """Clean and transform startup data."""
    df_clean = df.copy()
    
    # Remove duplicates
    df_clean = df_clean.drop_duplicates(subset=['name'], keep='first')
    
    # Handle missing values
    df_clean['city'] = df_clean['city'].fillna('Unknown')
    df_clean['employee_count'] = df_clean['employee_count'].fillna(0).astype(int)
    
    # Convert industry string to list
    df_clean['industry'] = df_clean['industry'].apply(
        lambda x: [i.strip() for i in x.split(',')] if pd.notna(x) else []
    )
    
    # Ensure founded_year is integer
    df_clean['founded_year'] = df_clean['founded_year'].astype(int)
    
    # Ensure total_funding_usd is numeric
    df_clean['total_funding_usd'] = pd.to_numeric(df_clean['total_funding_usd'], errors='coerce').fillna(0)
    
    print(f"✅ Cleaned {len(df_clean)} records")
    return df_clean

df_clean = clean_startup_data(df)
df_clean.head()

## 4. Transform to MongoDB Format

Convert DataFrame to list of dictionaries suitable for MongoDB insertion

In [ ]:
def transform_to_mongodb_format(df):
    """Transform DataFrame to MongoDB document format."""
    documents = []
    
    for _, row in df.iterrows():
        doc = {
            'name': row['name'],
            'founded_year': int(row['founded_year']),
            'country': row['country'],
            'city': row['city'],
            'industry': row['industry'],
            'total_funding_usd': float(row['total_funding_usd']),
            'employee_count': int(row['employee_count']),
            'status': row['status']
        }
        
        # Add optional fields if available
        if 'funding_rounds' in row and pd.notna(row['funding_rounds']):
            doc['funding_rounds'] = row['funding_rounds']
        
        if 'investors' in row and pd.notna(row['investors']):
            doc['investors'] = row['investors']
        
        documents.append(doc)
    
    print(f"✅ Transformed {len(documents)} documents for MongoDB")
    return documents

mongodb_documents = transform_to_mongodb_format(df_clean)

# Display sample document
print("\n📄 Sample MongoDB Document:")
print(json.dumps(mongodb_documents[0], indent=2))

## 5. Save Processed Data

In [ ]:
# Save to JSON for MongoDB ingestion
output_path = os.path.join(config.PROCESSED_DATA_DIR, 'cleaned_startups.json')

with open(output_path, 'w') as f:
    json.dump(mongodb_documents, f, indent=2)

print(f"✅ Saved {len(mongodb_documents)} documents to {output_path}")

# Also save as CSV for reference
csv_output_path = os.path.join(config.PROCESSED_DATA_DIR, 'cleaned_startups.csv')
df_clean.to_csv(csv_output_path, index=False)
print(f"✅ Saved CSV to {csv_output_path}")

## 6. Data Quality Summary

In [ ]:
print("📊 Data Quality Summary:")
print(f"\n✅ Total records processed: {len(df_clean)}")
print(f"✅ Unique companies: {df_clean['name'].nunique()}")
print(f"✅ Countries represented: {df_clean['country'].nunique()}")
print(f"✅ Industries covered: {sum(len(industries) for industries in df_clean['industry'])}")
print(f"✅ Date range: {df_clean['founded_year'].min()} - {df_clean['founded_year'].max()}")
print(f"\n💰 Total funding across all startups: ${df_clean['total_funding_usd'].sum():,.0f}")
print(f"👥 Total employees: {df_clean['employee_count'].sum():,}")

## Next Steps

1. ✅ Data has been cleaned and transformed
2. ✅ Documents are ready for MongoDB ingestion
3. 📥 Use `mongodb_setup.py` to insert data into MongoDB Atlas
4. 📊 Proceed to `analysis.ipynb` for exploratory data analysis